<a href="https://colab.research.google.com/github/Swleiii/Coursework/blob/main/Touchscreen_computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep Codes

In [ ]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle

In [ ]:
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "LA303"
PASSWORD = "HK12"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)

c = conn.cursor()


In [ ]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact, Text, DatePicker, IntText, Label, Layout, Button, Box, GridspecLayout, Checkbox, Output
from datetime import datetime
from IPython.display import display, clear_output

# Chef Tablet

In [ ]:
tb_all = TabBar(['Orders Dashboard','Submitted Orders','Orders Being Cooked','Completed Orders','Cancelled Orders','Message Box'])
def showPage(tabName):
  with tb_all.output_to(tabName):
    display()

####Orders Dashboard#####
with tb_all.output_to('Orders Dashboard'):
  #################################
  #######TABLE 1 ORDERS CODES######
  #################################
  T1_label = Label("TABLE ONE ORDERS")
  T1_notice_textbox = Text(description='Notice:', disabled=True)
  spacer = Label("----------------------------------------------------------------------------------------------")
  T1_order_textbox = Text(description='Order No:', disabled=True)
  T1_transaction_textbox = Text(description='Transaction No:',disabled=True)
  T1_table_textbox = Text(description='Table No:', disabled=True)
  T1_persons_textbox = Text(description='No of Persons:',disabled=True)
  T1_submission_time_textbox = Text(description='Submission Time:', disabled=True)
  T1_status_textbox = Text(description='Status:',disabled=True)

  T1_notice_textbox.style.description_width ='initial'
  T1_order_textbox.style.description_width = 'initial'
  T1_transaction_textbox.style.description_width = 'initial'
  T1_table_textbox.style.description_width = 'initial'
  T1_persons_textbox.style.description_width = 'initial'
  T1_submission_time_textbox.style.description_width = 'initial'
  T1_status_textbox.style.description_width = 'inital'

  T1_accept_button = widgets.Button(description="Accept")
  T1_serve_button = widgets.Button(description="Serve", disabled=True)
  T1_cancel_button = widgets.Button(description="Cancel")

  T1_top = widgets.VBox([T1_label,spacer, T1_notice_textbox, spacer,T1_order_textbox,T1_transaction_textbox,T1_table_textbox, T1_persons_textbox,T1_submission_time_textbox, spacer])
  T1_bottom_HBox = widgets.HBox([T1_accept_button,T1_serve_button,T1_cancel_button])
  T1_bottom = widgets.VBox([spacer,T1_status_textbox, T1_bottom_HBox])

  def T1_retrieve_textbox_value():
    c.execute("SELECT COOKING_ORDER.CO_ID, COOKING_ORDER.CO_QTY, COOKING_ORDER.CO_STATUS, COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, MENU.F_NAME, TRANSACTION.TR_HEADCOUNT, TRANSACTION.TABLE_TAB_ID FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE TRANSACTION.TABLE_TAB_ID = 1 AND COOKING_ORDER.CO_STATUS = 'Submitted'")
    row = c.fetchone()
    if row is not None:
      T1_order_textbox.value = str(row[0])
      T1_transaction_textbox.value = str(row[4])
      T1_table_textbox.value = str(row[7])
      T1_persons_textbox.value = str(row[6])
      T1_submission_time_textbox.value = str(row[3])
      T1_status_textbox.value = str(row[2])
      T1_notice_textbox.value =("NEW ORDER")
      T1_accept_button.disabled = False
    else:
      T1_notice_textbox.value = ("NO CURRENT ORDER")
      T1_order_textbox.value = ''
      T1_transaction_textbox.value = ''
      T1_table_textbox.value = ''
      T1_persons_textbox.value = ''
      T1_submission_time_textbox.value = ''
      T1_status_textbox.value = ''
      T1_accept_button.disabled = True
      T1_cancel_button.disabled = True

  def T1_foodtocook():
    c.execute("SELECT COOKING_ORDER.CO_QTY, MENU.F_NAME FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID WHERE COOKING_ORDER.CO_ID = '{}'".format(T1_order_textbox.value))
    food = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    T1_foodtocook.df = pd.DataFrame(food,columns = column_names)
    T1_foodtocook.df = T1_foodtocook.df.rename(columns={'CO_QTY': 'Quantity', 'F_NAME': 'Food Item'})
    T1_foodtocook.df.index = T1_foodtocook.df.index + 1
    return T1_foodtocook.df

  def T1_refresh_status():
    c.execute("""SELECT CO_STATUS FROM COOKING_ORDER WHERE CO_ID = '{}'""".format(T1_order_textbox.value))
    status = c.fetchone()
    T1_status_textbox.value = str(status[0])

  def on_T1_accept_button_clicked(b):
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cooking' WHERE CO_ID ='{}'""".format(T1_order_textbox.value))
    c.execute("commit")
    T1_notice_textbox.value =("ORDER COOKING")
    T1_refresh_status()
    T1_accept_button.disabled = True
    T1_serve_button.disabled = False
  T1_accept_button.on_click(on_T1_accept_button_clicked)

  def on_T1_serve_button_clicked(b):
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Completed', CO_READY_TIME = SYSDATE WHERE CO_ID = '{}' """.format(T1_order_textbox.value))
    c.execute("commit")
    T1_notice_textbox.value = ("COMPLETED ORDER")
    T1_retrieve_textbox_value()
    T1_foodtocook.df.drop
  T1_serve_button.on_click(on_T1_serve_button_clicked)

  def on_T1_cancel_button_clicked(b):
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cancelled' WHERE CO_ID ='{}'""".format(T1_order_textbox.value))
    c.execute("commit")
    T1_notice_textbox.value = ("ORDER CANCELLED")
    T1_retrieve_textbox_value()
    messageBox_label.value = "Please inform the staff about the cancellation"
    showPage("Message Box")
  T1_cancel_button.on_click(on_T1_cancel_button_clicked)

  #################################
  #######TABLE 2 ORDERS CODES######
  #################################
  T2_label = Label("TABLE TWO ORDERS")
  T2_notice_textbox = Text(description='Notice:', disabled=True)
  spacer = Label("----------------------------------------------------------------------------------------------")
  T2_order_textbox = Text(description='Order No:', disabled=True)
  T2_transaction_textbox = Text(description='Transaction No:',disabled=True)
  T2_table_textbox = Text(description='Table No:', disabled=True)
  T2_persons_textbox = Text(description='No of Persons:',disabled=True)
  T2_submission_time_textbox = Text(description='Submission Time:', disabled=True)
  T2_status_textbox = Text(description='Status:',disabled=True)

  T2_notice_textbox.style.description_width ='initial'
  T2_order_textbox.style.description_width = 'initial'
  T2_transaction_textbox.style.description_width = 'initial'
  T2_table_textbox.style.description_width = 'initial'
  T2_persons_textbox.style.description_width = 'initial'
  T2_submission_time_textbox.style.description_width = 'initial'
  T2_status_textbox.style.description_width = 'inital'

  T2_accept_button = widgets.Button(description="Accept")
  T2_serve_button = widgets.Button(description="Serve", disabled=True)
  T2_cancel_button = widgets.Button(description="Cancel")

  T2_top = widgets.VBox([T2_label,spacer,T2_notice_textbox, spacer,T2_order_textbox,T2_transaction_textbox,T2_table_textbox, T2_persons_textbox,T2_submission_time_textbox, spacer])
  T2_bottom_HBox = widgets.HBox([T2_accept_button,T2_serve_button,T2_cancel_button])
  T2_bottom = widgets.VBox([spacer,T2_status_textbox, T2_bottom_HBox])

  def T2_retrieve_textbox_value():
    c.execute("SELECT COOKING_ORDER.CO_ID, COOKING_ORDER.CO_QTY, COOKING_ORDER.CO_STATUS, COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, MENU.F_NAME, TRANSACTION.TR_HEADCOUNT, TRANSACTION.TABLE_TAB_ID FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE TRANSACTION.TABLE_TAB_ID = 2 AND COOKING_ORDER.CO_STATUS = 'Submitted'")
    row = c.fetchone()
    if row is not None:
      T2_order_textbox.value = str(row[0])
      T2_transaction_textbox.value = str(row[4])
      T2_table_textbox.value = str(row[7])
      T2_persons_textbox.value = str(row[6])
      T2_submission_time_textbox.value = str(row[3])
      T2_status_textbox.value = str(row[2])
      T2_notice_textbox.value =("NEW ORDER")
      T2_accept_button.disabled = False
    else:
      T2_notice_textbox.value = ("NO CURRENT ORDER")
      T2_order_textbox.value = ''
      T2_transaction_textbox.value = ''
      T2_table_textbox.value = ''
      T2_persons_textbox.value = ''
      T2_submission_time_textbox.value = ''
      T2_status_textbox.value = ''
      T2_accept_button.disabled = True
      T2_cancel_button.disabled = True

  def T2_foodtocook():
    c.execute("SELECT COOKING_ORDER.CO_QTY, MENU.F_NAME FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID WHERE COOKING_ORDER.CO_ID = '{}'".format(T2_order_textbox.value))
    food = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    T2_df = pd.DataFrame(food,columns = column_names)
    T2_df = T2_df.rename(columns={'CO_QTY': 'Quantity', 'F_NAME': 'Food Item'})
    T2_df.index = T2_df.index + 1
    return T2_df

  def T2_refresh_status():
    c.execute("""SELECT CO_STATUS FROM COOKING_ORDER WHERE CO_ID = '{}'""".format(T2_order_textbox.value))
    status = c.fetchone()
    T2_status_textbox.value = str(status[0])

  def on_T2_accept_button_clicked(b):
    #updates database
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cooking' WHERE CO_ID ='{}'""".format(T2_order_textbox.value))
    c.execute("commit")
    T2_notice_textbox.value =("ORDER COOKING")
    T2_refresh_status()
    T2_accept_button.disabled = True
    T2_serve_button.disabled = False
  T2_accept_button.on_click(on_T2_accept_button_clicked)

  def on_T2_serve_button_clicked(b):
    #updates database
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Completed', CO_READY_TIME = SYSDATE WHERE CO_ID = '{}' """.format(T2_order_textbox.value))
    c.execute("commit")
    T2_notice_textbox.value = ("COMPLETED ORDER")
    T2_retrieve_textbox_value()
  T2_serve_button.on_click(on_T2_serve_button_clicked)

  def on_T2_cancel_button_clicked(b):
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cancelled' WHERE CO_ID ='{}'""".format(T2_order_textbox.value))
    c.execute("commit")
    T2_notice_textbox.value = ("ORDER CANCELLED")
    T2_retrieve_textbox_value()
    messageBox_label.value = "Please inform the staff about the cancellation"
    showPage("Message Box")
  T2_cancel_button.on_click(on_T2_cancel_button_clicked)

  #################################
  #######TABLE 3 ORDERS CODES######
  #################################
  T3_label = Label("TABLE THREE ORDERS")
  T3_notice_textbox = Text(description='Notice:', disabled=True)
  spacer = Label("----------------------------------------------------------------------------------------------")
  T3_order_textbox = Text(description='Order No:', disabled=True)
  T3_transaction_textbox = Text(description='Transaction No:',disabled=True)
  T3_table_textbox = Text(description='Table No:', disabled=True)
  T3_persons_textbox = Text(description='No of Persons:',disabled=True)
  T3_submission_time_textbox = Text(description='Submission Time:', disabled=True)
  T3_status_textbox = Text(description='Status:',disabled=True)

  T3_notice_textbox.style.description_width ='initial'
  T3_order_textbox.style.description_width = 'initial'
  T3_transaction_textbox.style.description_width = 'initial'
  T3_table_textbox.style.description_width = 'initial'
  T3_persons_textbox.style.description_width = 'initial'
  T3_submission_time_textbox.style.description_width = 'initial'
  T3_status_textbox.style.description_width = 'inital'

  T3_accept_button = widgets.Button(description="Accept")
  T3_serve_button = widgets.Button(description="Serve", disabled=True)
  T3_cancel_button = widgets.Button(description="Cancel")

  T3_top = widgets.VBox([T3_label,spacer,T3_notice_textbox, spacer,T3_order_textbox,T3_transaction_textbox,T3_table_textbox, T3_persons_textbox,T3_submission_time_textbox, spacer])
  T3_bottom_HBox = widgets.HBox([T3_accept_button,T3_serve_button,T3_cancel_button])
  T3_bottom = widgets.VBox([spacer,T3_status_textbox, T3_bottom_HBox])

  def T3_retrieve_textbox_value():
    c.execute("SELECT COOKING_ORDER.CO_ID, COOKING_ORDER.CO_QTY, COOKING_ORDER.CO_STATUS, COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, MENU.F_NAME, TRANSACTION.TR_HEADCOUNT, TRANSACTION.TABLE_TAB_ID FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE TRANSACTION.TABLE_TAB_ID = 3 AND COOKING_ORDER.CO_STATUS = 'Submitted'")
    row = c.fetchone()
    if row is not None:
      T3_order_textbox.value = str(row[0])
      T3_transaction_textbox.value = str(row[4])
      T3_table_textbox.value = str(row[7])
      T3_persons_textbox.value = str(row[6])
      T3_submission_time_textbox.value = str(row[3])
      T3_status_textbox.value = str(row[2])
      T3_notice_textbox.value =("NEW ORDER")
      T3_accept_button.disabled = False
    else:
      T3_notice_textbox.value = ("NO CURRENT ORDER")
      T3_order_textbox.value = ''
      T3_transaction_textbox.value = ''
      T3_table_textbox.value = ''
      T3_persons_textbox.value = ''
      T3_submission_time_textbox.value = ''
      T3_status_textbox.value = ''
      T3_accept_button.disabled = True
      T3_cancel_button.disabled = True

  def T3_foodtocook():
    c.execute("SELECT COOKING_ORDER.CO_QTY, MENU.F_NAME FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID WHERE COOKING_ORDER.CO_ID = '{}'".format(T3_order_textbox.value))
    food = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    T3_df = pd.DataFrame(food,columns = column_names)
    T3_df = T3_df.rename(columns={'CO_QTY': 'Quantity', 'F_NAME': 'Food Item'})
    T3_df.index = T3_df.index + 1
    return T3_df

    def on_T3_serve_button_clicked(b):
    #updates database
      c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Completed', CO_READY_TIME = SYSDATE WHERE CO_ID = '{}' """.format(T3_order_textbox.value))
      c.execute("commit")
      T3_notice_textbox.value = ("COMPLETED ORDER")
      T3_retrieve_textbox_value()
      T3_foodtocook.df.drop
    T3_serve_button.on_click(on_T3_serve_button_clicked)

  def T3_refresh_status():
    c.execute("""SELECT CO_STATUS FROM COOKING_ORDER WHERE CO_ID = '{}'""".format(T3_order_textbox.value))
    status = c.fetchone()
    T3_status_textbox.value = str(status[0])

  def on_T3_accept_button_clicked(b):
    #updates database
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cooking' WHERE CO_ID ='{}'""".format(T3_order_textbox.value))
    c.execute("commit")
    T3_notice_textbox.value =("ORDER COOKING")
    T3_refresh_status()
    T3_accept_button.disabled = True
    T3_serve_button.disabled = False
  T3_accept_button.on_click(on_T3_accept_button_clicked)



  def on_T3_cancel_button_clicked(b):
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cancelled' WHERE CO_ID ='{}'""".format(T3_order_textbox.value))
    c.execute("commit")
    T3_notice_textbox.value = ("ORDER CANCELLED")
    T3_retrieve_textbox_value()
    messageBox_label.value = "Please inform the staff about the cancellation"
    showPage("Message Box")
  T3_cancel_button.on_click(on_T3_cancel_button_clicked)

  #################################
  #######TABLE 4 ORDERS CODES######
  #################################
  T4_label = Label("TABLE FOUR ORDERS")
  T4_notice_textbox = Text(description='Notice:', disabled=True)
  spacer = Label("----------------------------------------------------------------------------------------------")
  T4_order_textbox = Text(description='Order No:', disabled=True)
  T4_transaction_textbox = Text(description='Transaction No:',disabled=True)
  T4_table_textbox = Text(description='Table No:', disabled=True)
  T4_persons_textbox = Text(description='No of Persons:',disabled=True)
  T4_submission_time_textbox = Text(description='Submission Time:', disabled=True)
  T4_status_textbox = Text(description='Status:',disabled=True)

  T4_notice_textbox.style.description_width ='initial'
  T4_order_textbox.style.description_width = 'initial'
  T4_transaction_textbox.style.description_width = 'initial'
  T4_table_textbox.style.description_width = 'initial'
  T4_persons_textbox.style.description_width = 'initial'
  T4_submission_time_textbox.style.description_width = 'initial'
  T4_status_textbox.style.description_width = 'inital'

  T4_accept_button = widgets.Button(description="Accept")
  T4_serve_button = widgets.Button(description="Serve", disabled=True)
  T4_cancel_button = widgets.Button(description="Cancel")

  T4_top = widgets.VBox([T4_label,spacer,T4_notice_textbox, spacer,T4_order_textbox,T4_transaction_textbox,T4_table_textbox, T4_persons_textbox,T4_submission_time_textbox, spacer])
  T4_bottom_HBox = widgets.HBox([T4_accept_button,T4_serve_button,T4_cancel_button])
  T4_bottom = widgets.VBox([spacer,T4_status_textbox, T4_bottom_HBox])

  def T4_retrieve_textbox_value():
    c.execute("SELECT COOKING_ORDER.CO_ID, COOKING_ORDER.CO_QTY, COOKING_ORDER.CO_STATUS, COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, MENU.F_NAME, TRANSACTION.TR_HEADCOUNT, TRANSACTION.TABLE_TAB_ID FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE TRANSACTION.TABLE_TAB_ID = 4 AND COOKING_ORDER.CO_STATUS = 'Submitted'")
    row = c.fetchone()
    if row is not None:
      T4_order_textbox.value = str(row[0])
      T4_transaction_textbox.value = str(row[4])
      T4_table_textbox.value = str(row[7])
      T4_persons_textbox.value = str(row[6])
      T4_submission_time_textbox.value = str(row[3])
      T4_status_textbox.value = str(row[2])
      T4_notice_textbox.value =("NEW ORDER")
      T4_accept_button.disabled = False
    else:
      T4_notice_textbox.value = ("NO CURRENT ORDER")
      T4_order_textbox.value = ''
      T4_transaction_textbox.value = ''
      T4_table_textbox.value = ''
      T4_persons_textbox.value = ''
      T4_submission_time_textbox.value = ''
      T4_status_textbox.value = ''
      T4_accept_button.disabled = True
      T4_cancel_button.disabled = True

  def T4_foodtocook():
    c.execute("SELECT COOKING_ORDER.CO_QTY, MENU.F_NAME FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID WHERE COOKING_ORDER.CO_ID = '{}'".format(T4_order_textbox.value))
    food = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    T4_df = pd.DataFrame(food,columns = column_names)
    T4_df = T4_df.rename(columns={'CO_QTY': 'Quantity', 'F_NAME': 'Food Item'})
    T4_df.index = T4_df.index + 1
    return T4_df

  def T4_refresh_status():
    c.execute("""SELECT CO_STATUS FROM COOKING_ORDER WHERE CO_ID = '{}'""".format(T4_order_textbox.value))
    status = c.fetchone()
    T4_status_textbox.value = str(status[0])

  def on_T4_accept_button_clicked(b):
    #updates database
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cooking' WHERE CO_ID ='{}'""".format(T4_order_textbox.value))
    c.execute("commit")
    T4_notice_textbox.value =("ORDER COOKING")
    T4_refresh_status()
    T4_accept_button.disabled = True
    T4_serve_button.disabled = False
  T4_accept_button.on_click(on_T4_accept_button_clicked)

  def on_T4_serve_button_clicked(b):
    #updates database
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Completed', CO_READY_TIME = SYSDATE WHERE CO_ID = '{}' """.format(T4_order_textbox.value))
    c.execute("commit")
    T4_notice_textbox.value = ("COMPLETED ORDER")
    T4_retrieve_textbox_value()
  T4_serve_button.on_click(on_T4_serve_button_clicked)

  def on_T4_cancel_button_clicked(b):
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cancelled' WHERE CO_ID ='{}'""".format(T4_order_textbox.value))
    c.execute("commit")
    T4_notice_textbox.value = ("ORDER CANCELLED")
    T4_retrieve_textbox_value()
    messageBox_label.value = "Please inform the staff about the cancellation"
    showPage("Message Box")
  T4_cancel_button.on_click(on_T4_cancel_button_clicked)

  #################################
  #######TABLE 5 ORDERS CODES######
  #################################
  T5_label = Label("TABLE FIVE ORDERS")
  T5_notice_textbox = Text(description='Notice:', disabled=True)
  spacer = Label("----------------------------------------------------------------------------------------------")
  T5_order_textbox = Text(description='Order No:', disabled=True)
  T5_transaction_textbox = Text(description='Transaction No:',disabled=True)
  T5_table_textbox = Text(description='Table No:', disabled=True)
  T5_persons_textbox = Text(description='No of Persons:',disabled=True)
  T5_submission_time_textbox = Text(description='Submission Time:', disabled=True)
  T5_status_textbox = Text(description='Status:',disabled=True)

  T5_notice_textbox.style.description_width ='initial'
  T5_order_textbox.style.description_width = 'initial'
  T5_transaction_textbox.style.description_width = 'initial'
  T5_table_textbox.style.description_width = 'initial'
  T5_persons_textbox.style.description_width = 'initial'
  T5_submission_time_textbox.style.description_width = 'initial'
  T5_status_textbox.style.description_width = 'inital'

  T5_accept_button = widgets.Button(description="Accept")
  T5_serve_button = widgets.Button(description="Serve", disabled=True)
  T5_cancel_button = widgets.Button(description="Cancel")

  T5_top = widgets.VBox([T5_label,spacer,T5_notice_textbox, spacer,T5_order_textbox,T5_transaction_textbox,T5_table_textbox, T5_persons_textbox,T5_submission_time_textbox, spacer])
  T5_bottom_HBox = widgets.HBox([T5_accept_button,T5_serve_button,T5_cancel_button])
  T5_bottom = widgets.VBox([spacer,T5_status_textbox, T5_bottom_HBox])

  def T5_retrieve_textbox_value():
    c.execute("SELECT COOKING_ORDER.CO_ID, COOKING_ORDER.CO_QTY, COOKING_ORDER.CO_STATUS, COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, MENU.F_NAME, TRANSACTION.TR_HEADCOUNT, TRANSACTION.TABLE_TAB_ID FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE TRANSACTION.TABLE_TAB_ID = 5 AND COOKING_ORDER.CO_STATUS = 'Submitted'")
    row = c.fetchone()
    if row is not None:
      T5_order_textbox.value = str(row[0])
      T5_transaction_textbox.value = str(row[4])
      T5_table_textbox.value = str(row[7])
      T5_persons_textbox.value = str(row[6])
      T5_submission_time_textbox.value = str(row[3])
      T5_status_textbox.value = str(row[2])
      T5_notice_textbox.value =("NEW ORDER")
      T5_accept_button.disabled = False
    else:
      T5_notice_textbox.value = ("NO CURRENT ORDER")
      T5_order_textbox.value = ''
      T5_transaction_textbox.value = ''
      T5_table_textbox.value = ''
      T5_persons_textbox.value = ''
      T5_submission_time_textbox.value = ''
      T5_status_textbox.value = ''
      T5_accept_button.disabled = True
      T5_cancel_button.disabled = True

  def T5_foodtocook():
    c.execute("SELECT COOKING_ORDER.CO_QTY, MENU.F_NAME FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID WHERE COOKING_ORDER.CO_ID = '{}'".format(T5_order_textbox.value))
    food = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    T5_df = pd.DataFrame(food,columns = column_names)
    T5_df = T5_df.rename(columns={'CO_QTY': 'Quantity', 'F_NAME': 'Food Item'})
    T5_df.index = T5_df.index + 1
    # T5_df.set_index('Quantity',inplace=True)
    # T5_df.index = df.index + 1
    return T5_df

  def T5_refresh_status():
    c.execute("""SELECT CO_STATUS FROM COOKING_ORDER WHERE CO_ID = '{}'""".format(T5_order_textbox.value))
    status = c.fetchone()
    T5_status_textbox.value = str(status[0])

  def on_T5_accept_button_clicked(b):
    #updates database
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cooking' WHERE CO_ID ='{}'""".format(T5_order_textbox.value))
    c.execute("commit")
    T5_notice_textbox.value =("ORDER COOKING")
    T5_refresh_status()
    T5_accept_button.disabled = True
    T5_serve_button.disabled = False
  T5_accept_button.on_click(on_T5_accept_button_clicked)

  def on_T5_serve_button_clicked(b):
    #updates database
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Completed', CO_READY_TIME = SYSDATE WHERE CO_ID = '{}' """.format(T5_order_textbox.value))
    c.execute("commit")
    T5_notice_textbox.value = ("COMPLETED ORDER")
    T5_retrieve_textbox_value()
  T5_serve_button.on_click(on_T5_serve_button_clicked)

  def on_T5_cancel_button_clicked(b):
    c.execute("""UPDATE COOKING_ORDER SET CO_STATUS = 'Cancelled' WHERE CO_ID ='{}'""".format(T5_order_textbox.value))
    c.execute("commit")
    messageBox_label.value = "Please inform the staff about the cancellation"
    showPage("Message Box")
    T5_retrieve_textbox_value()
  T5_cancel_button.on_click(on_T5_cancel_button_clicked)

  #################################
  ####### GRID FORMATTING #########
  #################################
  grid = GridspecLayout(2, 3, grid_gap='5px')

  def table_one():
    T1_retrieve_textbox_value()
    food_to_cook = T1_foodtocook()
    T1_display = Output()
    with T1_display:
      display(food_to_cook)
    T1_layout = widgets.VBox([T1_top,T1_display,T1_bottom])
    return T1_layout

  def table_two():
    T2_retrieve_textbox_value()
    food_to_cook = T2_foodtocook()
    T2_display = Output()
    with T2_display:
      display(food_to_cook)
    T2_layout = widgets.VBox([T2_top,T2_display,T2_bottom])
    return T2_layout

  def table_three():
    T3_retrieve_textbox_value()
    food_to_cook = T3_foodtocook()
    T3_display = Output()
    with T3_display:
      display(food_to_cook)
    T3_layout = widgets.VBox([T3_top,T3_display,T3_bottom])
    return T3_layout

  def table_four():
    T4_retrieve_textbox_value()
    food_to_cook = T4_foodtocook()
    T4_display = Output()
    with T4_display:
      display(food_to_cook)
    T4_layout = widgets.VBox([T4_top,T4_display,T4_bottom])
    return T4_layout

  def table_five():
    T5_retrieve_textbox_value()
    food_to_cook = T5_foodtocook()
    T5_display = Output()
    with T5_display:
      display(food_to_cook)
    T5_layout = widgets.VBox([T5_top,T5_display,T5_bottom])
    return T5_layout


  grid[0, 0] = Box([table_one()], layout=Layout(width='430px', height='520px', border='1px solid black'))
  grid[0, 1] = Box([table_two()], layout=Layout(width='430px', height='520px', border='1px solid black'))
  grid[0, 2] = Box([table_three()], layout=Layout(width='430px', height='520px', border='1px solid black'))
  grid[1, 0] = Box([table_four()], layout=Layout(width='430px', height='520px', border='1px solid black'))
  grid[1, 1] = Box([table_five()], layout=Layout(width='430px', height='520px', border='1px solid black'))

  display(grid)


###inital dataframes ###
with tb_all.output_to('Submitted Orders'):
  refresh_submitted_orders_button = widgets.Button(description="REFRESH")
  c.execute("SELECT COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, COOKING_ORDER.CO_ID, TRANSACTION.TABLE_TAB_ID, TRANSACTION.TR_HEADCOUNT, MENU.F_NAME,COOKING_ORDER.CO_QTY,  COOKING_ORDER.CO_STATUS FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE COOKING_ORDER.CO_STATUS='Submitted'")
  table = c.fetchall()
  column_names = [colname[0] for colname in c.description]
  df = pd.DataFrame(table,columns = column_names)
  df = df.rename(columns={'CO_SUBMISSION_TIME': 'Submission Time', 'TRANSACTION_TR_ID':'Transaction ID', 'CO_ID': 'Cooking Order ID', 'TABLE_TAB_ID': 'Table No','TR_HEADCOUNT':'Headcount', 'F_NAME': 'Food Item', 'CO_QTY': 'Quantity', 'CO_STATUS': 'Order Status'})
  df = df.sort_values(by='Submission Time',ascending=False)
  df.reset_index(drop=True, inplace=True)
  df.index = df.index + 1
  display(refresh_submitted_orders_button)
  display(df)

with tb_all.output_to('Orders Being Cooked'):
  refresh_accepted_orders_button = widgets.Button(description="REFRESH")
  c.execute("SELECT COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, COOKING_ORDER.CO_ID, TRANSACTION.TABLE_TAB_ID, TRANSACTION.TR_HEADCOUNT, MENU.F_NAME,COOKING_ORDER.CO_QTY,  COOKING_ORDER.CO_STATUS FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE COOKING_ORDER.CO_STATUS='Cooking'")
  table = c.fetchall()
  column_names = [colname[0] for colname in c.description]
  df = pd.DataFrame(table,columns = column_names)
  df = df.rename(columns={'CO_SUBMISSION_TIME': 'Submission Time', 'TRANSACTION_TR_ID':'Transaction ID', 'CO_ID': 'Cooking Order ID', 'TABLE_TAB_ID': 'Table No','TR_HEADCOUNT':'Headcount', 'F_NAME': 'Food Item', 'CO_QTY': 'Quantity', 'CO_STATUS': 'Order Status'})
  df = df.sort_values(by='Submission Time',ascending=False)
  df.reset_index(drop=True, inplace=True)
  df.index = df.index + 1
  display(refresh_accepted_orders_button)
  display(df)

with tb_all.output_to('Completed Orders'):
  refresh_completed_orders_button = widgets.Button(description="REFRESH")
  c.execute("SELECT COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.CO_READY_TIME, COOKING_ORDER.TRANSACTION_TR_ID, COOKING_ORDER.CO_ID, TRANSACTION.TABLE_TAB_ID, TRANSACTION.TR_HEADCOUNT, MENU.F_NAME,COOKING_ORDER.CO_QTY,  COOKING_ORDER.CO_STATUS FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE COOKING_ORDER.CO_STATUS='Completed'")
  table = c.fetchall()
  column_names = [colname[0] for colname in c.description]
  df = pd.DataFrame(table,columns = column_names)
  df = df.rename(columns={'CO_SUBMISSION_TIME': 'Submission Time', 'CO_READY_TIME': 'Time Completed', 'TRANSACTION_TR_ID':'Transaction ID', 'CO_ID': 'Cooking Order ID', 'TABLE_TAB_ID': 'Table No','TR_HEADCOUNT':'Headcount', 'F_NAME': 'Food Item', 'CO_QTY': 'Quantity', 'CO_STATUS': 'Order Status'})
  df = df.sort_values(by='Submission Time',ascending=False)
  df.reset_index(drop=True, inplace=True)
  df.index = df.index + 1
  display(refresh_completed_orders_button)
  display(df)

with tb_all.output_to('Cancelled Orders'):
  refresh_cancelled_orders_button = widgets.Button(description="REFRESH")
  c.execute("SELECT COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, COOKING_ORDER.CO_ID, TRANSACTION.TABLE_TAB_ID, TRANSACTION.TR_HEADCOUNT, MENU.F_NAME,COOKING_ORDER.CO_QTY,  COOKING_ORDER.CO_STATUS FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE COOKING_ORDER.CO_STATUS='Cancelled'")
  table = c.fetchall()
  column_names = [colname[0] for colname in c.description]
  df = pd.DataFrame(table,columns = column_names)
  df = df.rename(columns={'CO_SUBMISSION_TIME': 'Submission Time', 'TRANSACTION_TR_ID':'Transaction ID', 'CO_ID': 'Cooking Order ID', 'TABLE_TAB_ID': 'Table No','TR_HEADCOUNT':'Headcount', 'F_NAME': 'Food Item', 'CO_QTY': 'Quantity', 'CO_STATUS': 'Order Status'})
  df = df.sort_values(by='Submission Time',ascending=False)
  df.reset_index(drop=True, inplace=True)
  df.index = df.index + 1
  display(refresh_cancelled_orders_button)
  display(df)


#####Refresh Functions#####
def generate_submitted_orders():
  with tb_all.output_to('Submitted Orders'):
    tb_all.clear_tab()
    refresh_submitted_orders_button_new = widgets.Button(description="REFRESH")
    c.execute("SELECT COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, COOKING_ORDER.CO_ID, TRANSACTION.TABLE_TAB_ID, TRANSACTION.TR_HEADCOUNT, MENU.F_NAME,COOKING_ORDER.CO_QTY,  COOKING_ORDER.CO_STATUS FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE COOKING_ORDER.CO_STATUS='Submitted'")
    table = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    df = pd.DataFrame(table,columns = column_names)
    df = df.rename(columns={'CO_SUBMISSION_TIME': 'Submission Time', 'TRANSACTION_TR_ID':'Transaction ID', 'CO_ID': 'Cooking Order ID', 'TABLE_TAB_ID': 'Table No','TR_HEADCOUNT':'Headcount', 'F_NAME': 'Food Item', 'CO_QTY': 'Quantity', 'CO_STATUS': 'Order Status'})
    df = df.sort_values(by='Submission Time',ascending=False)
    df.reset_index(drop=True, inplace=True)
    df.index = df.index + 1
    display(refresh_submitted_orders_button)
    display(df)

    def on_refresh_submitted_orders_clicked_new(b):
      generate_submitted_orders()
    refresh_submitted_orders_button_new.on_click(on_refresh_submitted_orders_clicked_new)

def generate_accepted_orders():
  with tb_all.output_to('Orders Being Cooked'):
    tb_all.clear_tab()
    refresh_accepted_orders_button_new = widgets.Button(description="REFRESH")
    c.execute("SELECT COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, COOKING_ORDER.CO_ID, TRANSACTION.TABLE_TAB_ID, TRANSACTION.TR_HEADCOUNT, MENU.F_NAME,COOKING_ORDER.CO_QTY,  COOKING_ORDER.CO_STATUS FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE COOKING_ORDER.CO_STATUS='Cooking'")
    table = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    df = pd.DataFrame(table,columns = column_names)
    df = df.rename(columns={'CO_SUBMISSION_TIME': 'Submission Time', 'TRANSACTION_TR_ID':'Transaction ID', 'CO_ID': 'Cooking Order ID', 'TABLE_TAB_ID': 'Table No','TR_HEADCOUNT':'Headcount', 'F_NAME': 'Food Item', 'CO_QTY': 'Quantity', 'CO_STATUS': 'Order Status'})
    df = df.sort_values(by='Submission Time',ascending=False)
    df.reset_index(drop=True, inplace=True)
    df.index = df.index + 1
    display(refresh_accepted_orders_button)
    display(df)

    def on_refresh_accepted_orders_clicked_new(b):
      generate_accepted_orders()
    refresh_accepted_orders_button_new.on_click(on_refresh_accepted_orders_clicked_new)

####Completed Orders#####
def generate_completed_orders():
  with tb_all.output_to('Completed Orders'):
    tb_all.clear_tab()
    refresh_completed_orders_button_new = widgets.Button(description="REFRESH")
    c.execute("SELECT COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.CO_READY_TIME, COOKING_ORDER.TRANSACTION_TR_ID, COOKING_ORDER.CO_ID, TRANSACTION.TABLE_TAB_ID, TRANSACTION.TR_HEADCOUNT, MENU.F_NAME,COOKING_ORDER.CO_QTY,  COOKING_ORDER.CO_STATUS FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE COOKING_ORDER.CO_STATUS='Completed'")
    table = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    df = pd.DataFrame(table,columns = column_names)
    df = df.rename(columns={'CO_SUBMISSION_TIME': 'Submission Time', 'CO_READY_TIME': 'Time Completed', 'TRANSACTION_TR_ID':'Transaction ID', 'CO_ID': 'Cooking Order ID', 'TABLE_TAB_ID': 'Table No','TR_HEADCOUNT':'Headcount', 'F_NAME': 'Food Item', 'CO_QTY': 'Quantity', 'CO_STATUS': 'Order Status'})
    df = df.sort_values(by='Submission Time',ascending=False)
    df.reset_index(drop=True, inplace=True)
    df.index = df.index + 1
    display(refresh_completed_orders_button)
    display(df)

    def on_refresh_completed_orders_clicked_new(b):
      generate_completed_orders()
    refresh_completed_orders_button_new.on_click(on_refresh_completed_orders_clicked_new)

####Cancelled Orders#####
def generate_cancelled_orders():
  with tb_all.output_to('Cancelled Orders'):
    tb_all.clear_tab()
    refresh_cancelled_orders_button_new = widgets.Button(description="REFRESH")
    c.execute("SELECT COOKING_ORDER.CO_SUBMISSION_TIME, COOKING_ORDER.TRANSACTION_TR_ID, COOKING_ORDER.CO_ID, TRANSACTION.TABLE_TAB_ID, TRANSACTION.TR_HEADCOUNT, MENU.F_NAME,COOKING_ORDER.CO_QTY,  COOKING_ORDER.CO_STATUS FROM COOKING_ORDER INNER JOIN MENU ON COOKING_ORDER.MENU_F_ID = MENU.F_ID INNER JOIN TRANSACTION ON COOKING_ORDER.TRANSACTION_TR_ID = TRANSACTION.TR_ID WHERE COOKING_ORDER.CO_STATUS='Cancelled'")
    table = c.fetchall()
    column_names = [colname[0] for colname in c.description]
    df = pd.DataFrame(table,columns = column_names)
    df = df.rename(columns={'CO_SUBMISSION_TIME': 'Submission Time', 'TRANSACTION_TR_ID':'Transaction ID', 'CO_ID': 'Cooking Order ID', 'TABLE_TAB_ID': 'Table No','TR_HEADCOUNT':'Headcount', 'F_NAME': 'Food Item', 'CO_QTY': 'Quantity', 'CO_STATUS': 'Order Status'})
    df = df.sort_values(by='Submission Time', ascending=False)
    df.reset_index(drop=True, inplace=True)
    df.index = df.index + 1
    display(refresh_cancelled_orders_button)
    display(df)

    def on_refresh_cancelled_orders_clicked_new(b):
      generate_cancelled_orders()
    refresh_cancelled_orders_button_new.on_click(on_refresh_cancelled_orders_clicked_new)


### Button Functions ###
def on_refresh_submitted_orders_clicked(b):
  generate_submitted_orders()
refresh_submitted_orders_button.on_click(on_refresh_submitted_orders_clicked)

def on_refresh_accepted_orders_clicked(b):
  generate_accepted_orders()
refresh_accepted_orders_button.on_click(on_refresh_accepted_orders_clicked)

def on_refresh_completed_orders_clicked(b):
  generate_completed_orders()
refresh_completed_orders_button.on_click(on_refresh_completed_orders_clicked)

def on_refresh_cancelled_orders_clicked(b):
  generate_cancelled_orders()
refresh_cancelled_orders_button.on_click(on_refresh_cancelled_orders_clicked)

######Message Box#######
with tb_all.output_to('Message Box'):
  messageBox_label = Label("")
  ok_button = widgets.Button(description="OK")
  def on_ok_button_clicked(b):
    showPage('Orders Dashboard')
    messageBox_label = Label("")
  ok_button.on_click(on_ok_button_clicked)
  display(widgets.VBox([messageBox_label,ok_button]))

display(tb_all)